In [36]:
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import os
import subprocess
import numpy as np
import pandas as pd

In [71]:
imageDirectory = '/data/mj1e16/kepler/cal/'
diffDir = '/data/mj1e16/kepler/properDiff/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'

# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'
# imageDirectory = '/media/mj1e16/PP AV-TV/keplerCal/'

In [5]:
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0]!='.']
dirlist = [imageDirectory+x for x in dirlist]

In [6]:
supernovaStart = [56176.666,55846.320,55928.414]

In [7]:
def findClosestImage(supernovaStart,dirlist):
    answer = []
    for y in range(len(supernovaStart)):
        timediff = []
        for x in range(len(dirlist)):
            #print(dirlist[x])
            hdu = fits.open(dirlist[x])
            header = hdu[1].header
            timediff.append(abs(header['MJDSTART']-supernovaStart[y]))

        closest = min(timediff)
        imageIndex =  timediff.index(closest)   
        answer.append({'timediff':closest,'index':imageIndex,'image':dirlist[imageIndex]})
    return answer

In [8]:
answer = findClosestImage(supernovaStart,dirlist)

In [9]:
print(answer)

[{'index': 46, 'timediff': 7.854872060001071, 'image': '/data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits'}, {'index': 6, 'timediff': 13.539371960003336, 'image': '/data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits'}, {'index': 41, 'timediff': 2.427519040000334, 'image': '/data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits'}]


In [10]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [11]:
extensionOrders = []
for image in dirlist:
    extensionOrders.append(correctExtensionOrder(image))

In [24]:
def makeDS9RegFile(xcoords,ycoords,fileNameBase):
    
    bigString = 'global color=lightgreen\nWCS\n'
    bigString += 'circle({},{},0.005)\n'.format(xcoords,ycoords)
    bigString += 'circle({},{},0.1)\n'.format(xcoords,ycoords)
    fileName = fileNameBase+'.reg'
    with open(fileName,'w') as f:
        f.write(bigString)

In [13]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [42]:
def findMatchesNoCondition(tab,sneRa,sneDec,max_sep = 5 * u.arcsec):
    
    df = tab.to_pandas()
    
    number2 = tab['index'] # difference image
    ra2 = tab['RA']
    dec2 = tab['DEC']    
    
    cat1 = SkyCoord(ra=sneRa*u.degree, dec=sneDec*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['index'].isin(objectsFound)]
    matchTab = Table.from_pandas(matchdf)
    
    return matchTab

In [55]:
sneIms = ['kplr2012242195726_ffi-cal.fits[45]','kplr2011271191331_ffi-cal.fits[1]','kplr2012004204112_ffi-cal.fits[44]']

In [72]:
sneDiffs = ['diff_imNo46_channel37_ccd45.fits','diff_imNo6_channel81_ccd1.fits','diff_imNo41_channel41_ccd44.fits']

In [56]:
imageName = bigTab['IMAGE_NAME']
diffName = bigTab['DIFF_NAME']
sneDiffs = [y for x,y in zip(imageName,diffName) if x in sneIms]
print(len(sneDiffs))
setsneIms = set(sneDiffs)


81


In [57]:
bigTab = ascii.read('/home/mj1e16/outTablesImproved/firsthalf.csv')
imageName = bigTab['IMAGE_NAME']
diffName = bigTab['DIFF_NAME']
sneDiffs = [y for x,y in zip(imageName,diffName) if x in sneIms]
print(len(sneDiffs))
setsneIms = set(sneDiffs)
print(setsneIms)

183
set(['diff_imNo46_channel37_ccd45.fits', 'diff_imNo6_channel81_ccd1.fits'])


In [58]:
bigTab = ascii.read('/home/mj1e16/outTablesImproved/secondhalf.csv')
imageName = bigTab['IMAGE_NAME']
diffName = bigTab['DIFF_NAME']
sneDiffs = [y for x,y in zip(imageName,diffName) if x in sneIms]
print(len(sneDiffs))
setsneIms = set(sneDiffs)
print(setsneIms)

81
set(['diff_imNo41_channel41_ccd44.fits'])


In [70]:
df = bigTab.to_pandas()
newdf = df[df['IMAGE_NAME'].isin(sneIms)]
newnewdf= newdf[newdf['RA'] >290.0]
newnewnewdf= newnewdf[newnewdf['RA'] <300.0]

newnewdf.sort_values(['RA'])

,NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index
73499,34,1057.9450,15.09974,17.4388,236.6517,506.0450,0.240,290.027401,43.434450,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73499
73481,16,1987.5830,23.22541,16.7542,385.8151,712.6911,0.019,290.043386,43.715688,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73481
73495,30,859.9157,14.53681,17.6639,284.4750,548.8400,0.119,290.052215,43.503002,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73495
73506,41,763.0076,13.75045,17.7937,237.6670,461.2464,0.116,290.066256,43.393786,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73506
73530,65,1445.9030,20.00984,17.0997,38.5034,144.8911,0.319,290.078589,42.981034,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73530
73523,58,1378.5750,20.82691,17.1514,91.3558,224.6766,0.112,290.078805,43.086722,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73523
73545,80,892.2217,14.72685,17.6238,603.9863,982.4825,0.146,290.094554,44.097106,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73545
73498,33,3356.0570,26.36528,16.1854,298.0427,515.2161,0.156,290.097674,43.480265,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73498
73513,48,808.8060,15.64241,17.7304,226.5169,340.4728,0.005,290.153145,43.275657,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73513
73502,37,2074.8360,22.49563,16.7075,332.8059,498.3153,0.125,290.156033,43.485814,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73502


In [62]:
newdf

,NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index
73466,1,42102.5100,115.91130,13.4392,735.0262,108.4561,0.948,290.992504,43.367569,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73466
73467,2,15157.1300,114.85100,14.5485,747.5010,119.2660,0.944,290.999540,43.385067,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73467
73468,3,7199.3510,140.47520,15.3568,744.0814,49.6475,0.946,291.052443,43.318378,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73468
73469,4,8152.9010,136.79370,15.2217,639.6380,26.1076,0.405,290.938582,43.234043,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73469
73470,5,6953.6990,137.51780,15.3945,648.7786,24.3686,0.320,290.951659,43.237903,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73470
73471,6,1073.9360,18.86546,17.4226,865.9760,19.7982,0.016,291.232750,43.363357,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73471
73472,7,1330.5140,20.14832,17.1900,417.4348,34.3162,0.074,290.649336,43.108224,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73472
73473,8,3160.9950,90.66022,16.2504,744.7331,75.1668,0.735,291.032296,43.342466,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73473
73474,9,1308.4290,19.01228,17.2081,288.7361,55.9842,0.051,290.468366,43.050656,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73474
73475,10,2861.5110,44.36894,16.3585,707.8987,70.8549,0.537,290.988816,43.316433,kplr2012004204112_ffi-cal.fits[44],meidian_41_44.fits,diff_imNo41_channel41_ccd44.fits,73475


In [50]:
setsneIms = set(sneDiffs)
print(setsneIms)

set(['diff_imNo41_channel42_ccd41.fits'])


In [30]:
names = ['KSN 2012a','KSN 2011b','KSN 2011c'] 
# ccds = [61,42,29]
# ccds = [37,41,81]
#ccds = [21,44,63]
imges = ['kplr2012242195726_ffi-cal.fits','kplr2011271191331_ffi-cal.fits','kplr2012004204112_ffi-cal.fits']
ccds = [45,1,44]
ra = [293.3754458333333, 290.15641666666664, 291.1920374999999]
dec = [45.25035833333333, 38.252316666666665, 43.68074444444444]

In [60]:
#bigTab = ascii.read('/home/mj1e16/outTablesImproved/firsthalf.csv')
bigTab = ascii.read('/home/mj1e16/outTablesImproved/secondhalf.csv')
indexes = [i for i in range(len(bigTab))]
bigTab['index'] = indexes
sneTab = findMatchesNoCondition(bigTab,ra,dec)
print(len(sneTab))

0


In [44]:
bigTab = ascii.read('/home/mj1e16/outTablesImproved/secondhalf.csv')
indexes = [i for i in range(len(bigTab))]
bigTab['index'] = indexes
sneTab = findMatchesNoCondition(bigTab,ra,dec)
print(len(sneTab))

0


In [40]:
indexes = [i for i in range(len(bigTab))]
bigTab['index'] = indexes
sneTab = findMatchesNoCondition(bigTab,ra,dec)

In [41]:
sneTab

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64


In [15]:
makeDS9RegFile(ra,dec,'supernovae')

In [25]:
for x in range(len(names)):

    makeDS9RegFile(ra[x],dec[x],'supernovae')
    #print('ds9 '+answer[x]['image']+'[{}]'.format(extension)+' -regions supernovae{}.reg'.format(x))
    subprocess.call(['ds9',answer[x]['image']+'[{}]'.format(ccds[x]),'-regions','supernovae.reg'.format(x),'-scale','mode','zscale'])

In [ ]:
# ds9 /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -regions supernovae0.reg
# ds9 /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -regions supernovae1.reg
# ds9 /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[44] -regions supernovae2.reg

In [80]:
def hotpants(imNo,channel,extensionOrders,imagelist,name,medianDir='/data/mj1e16/kepler/medians/'):
    ccdChannel = int(extensionOrders[imNo][channel])
    inim = imagelist[imNo]+'[{}]'.format(ccdChannel)
    tmplim = medianDir+'meidian_{}.fits'.format(channel)
    print(tmplim)
#     outim = '/home/mj1e16/testImages/supernovaDiff_'+name+'.fits'
#     # outim = '/data/mj1e16/kepler/properDiff/diff_imNo{}_channel{}.fits'.format(imNo,channel)
#     subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])